In [1]:
from scipy.io import loadmat
import numpy as np
import torch 

monkey_data = loadmat('monkey_data.mat') 
visual_data = monkey_data['inp'][0]
muscle_data = monkey_data['targ'][0]
test = np.zeros((502, 2))
for i, data in enumerate(visual_data):
    test[i] = data.shape

print(test[:,1].max())
batch_size = visual_data.shape[0] # N, 502
trial_len = int(test[:,1].max()) # visual_data[0].shape[1] # T, 298 
in_dim = visual_data[0].shape[0] # L, 21
out_dim = muscle_data[0].shape[0] # O, 50

hid_dim = 100
num_layers = 3
new_visual_data = np.zeros((batch_size, trial_len, in_dim))

for i, data in enumerate(visual_data):
    new_visual_data[i, 0:visual_data[i].shape[1], :] = visual_data[i].transpose()
print(new_visual_data.shape)
import math
from sklearn.model_selection import train_test_split
from sklearn.utils import gen_batches
# np.random.seed()

 # shuffling was somehow change 
train_idx, test_idx = train_test_split(
     range(batch_size), 
     test_size=0.2, 
     shuffle=True )

train_input = np.zeros((int(batch_size * .8), trial_len, in_dim))
train_output = np.zeros((int(batch_size * .8), trial_len, out_dim))
test_input = np.zeros((int(np.ceil(batch_size * .2)), trial_len, in_dim))
test_output = np.zeros((int(np.ceil(batch_size * .2)), trial_len, out_dim))
train_ind = 0
test_ind = 0
for i, data in enumerate(visual_data):
     if i in train_idx:
       # train_input[train_ind, 0:visual_data[i].shape[1], :] = visual_data[i].transpose()
       train_input[train_ind, -visual_data[i].shape[1]:, :] = visual_data[i].transpose()
       train_output[train_ind, -visual_data[i].shape[1]:, :] = muscle_data[i].transpose()
       train_ind += 1
     else:
        test_input[test_ind, -visual_data[i].shape[1]:, :] = visual_data[i].transpose()
        test_output[test_ind, -visual_data[i].shape[1]:, :] = muscle_data[i].transpose()
        test_ind += 1

# for use in weighted loss
muscle_length_vec = torch.tensor([9.8, 10.8, 13.7, 6.8, 7.6, 8.7, 7.4, 16.2, 14.4, 13.8, 13.8, 25.4, 23.2, 27.9, 9.3, 13.4, 11.4, 11.4, 2.7, 3.3, 11.6, 13.2, 8.6, 17.3, 8.1, 5.9, 6.2, 6.3, 5.1, 6.4, 4.9, 2.8, 5.2, 7.4, 7.5, 8.4, 7.5, 8.0, 8.4, 7.5, 6.5, 6.3, 7.2, 7.0, 6.8, 5.9, 5.4, 6.8, 5.5, 7.1])
muscle_weight_vec = (1/muscle_length_vec) * (2.5*torch.min(muscle_length_vec))





341.0
(502, 341, 21)


In [9]:
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import pickle 
from tqdm import tqdm

from mrnn7 import MilliesRNN
from hessianfree import HessianFree

hessian = False
hardcore = True
intermodule_connections_removed = .9

learning_rates = [0.01, 0.001, 0.0001]
batch_size = [3,4,5]
weight_decay = [0,1e-6,1e-5,1e-4]

num_epochs = 5
test_losses = {}

print("When i = 0, clipping, when i = 1, no clipping\n")

criterion1 = nn.MSELoss() 

for l in tqdm(learning_rates, desc="Searching over models..."):
    for b in batch_size:
        for w in weight_decay:
            optimizer = torch.optim.Adam(model.parameters(), lr=l, weight_decay = w)
            for i in range(2):
                print("Learning rate = " + str(l) + ", batch size = " + str(1/b) + ", i = " + str(i) + " weight decay = " + str(w) + "\n")
                model_type = f"mse_{l}_{b}_{i}_{w}"
                model = MilliesRNN(in_dim, hid_dim, out_dim, True)
                module1 = model.h2o
                prune.random_unstructured(module1, name="weight", amount=intermodule_connections_removed)
                module2 = model.thal
                prune.random_unstructured(module2, name="weight", amount=intermodule_connections_removed)
                model.train()
                for epoch in range(num_epochs):
                    batches = []
                    shuffed = np.arange(0,401)
                    np.random.shuffle(shuffed)
                    for i in range(b):
                        curr_b_size = int(train_input.shape[0]/b)
                        idx = shuffed[(curr_b_size*i):curr_b_size*(i+1)]
                        
                        batches.append(idx)
                    
                    for j, batch in enumerate(batches):
                        
                        optimizer.zero_grad()

                        outputs = model(torch.from_numpy(train_input[batch]).to(torch.float32))   

                        loss = criterion1(outputs, torch.from_numpy(train_output[batch]).to(torch.float32))
                        
                        loss.backward()
                        optimizer.step()
                        
                        if(i == 0):
                            nn.utils.clip_grad_norm_(model.parameters(), 1)
                
                preds = torch.empty((curr_b_size, trial_len, out_dim))

                num_correct = 0
                num_samples = len(test_input)

                model.eval()
                with torch.no_grad():
                    out = model(torch.from_numpy(test_input).to(torch.float32))
                    loss1 = criterion1(out, torch.from_numpy(test_output).to(torch.float32), model.named_parameters())

                print(f"Average test: {loss1 / num_samples}\n")
                test_losses[model_type] = {loss1 / num_samples}

                with torch.no_grad():
                    out = model(torch.from_numpy(new_visual_data).to(torch.float32))

                with open(f'outputs/{model_type}.pickle', 'wb') as handle:
                    pickle.dump(out, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

When i = 0, clipping, when i = 1, no clipping



Searching over models...:   0%|          | 0/3 [00:00<?, ?it/s]

Learning rate = 0.01, batch size = 0.3333333333333333, i = 0 weight decay = 0



Searching over models...:   0%|          | 0/3 [01:15<?, ?it/s]


TypeError: empty(): argument 'size' (position 1) must be tuple of ints, but found element of type float at pos 0